In [168]:
import gym
import math
import numpy as np
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

c:\Users\Dell\miniconda3\envs\gyenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [169]:
from gym.envs import box2d

In [170]:
FPS = 50
SCALE = 30.0  # affects how fast-paced the game is, forces should be adjusted as well

MAIN_ENGINE_POWER = 13.0
SIDE_ENGINE_POWER = 0.6

INITIAL_RANDOM = 1000.0  # Set 1500 to make game harder

LANDER_POLY = [(-14, +17), (-17, 0), (-17, -10), (+17, -10), (+17, 0), (+14, +17)]
LEG_AWAY = 20
LEG_DOWN = 18
LEG_W, LEG_H = 2, 8
LEG_SPRING_TORQUE = 40

SIDE_ENGINE_HEIGHT = 14.0
SIDE_ENGINE_AWAY = 12.0

VIEWPORT_W = 600
VIEWPORT_H = 400

In [171]:
class RewardShaped(box2d.lunar_lander.LunarLander):
    def __init__(self, **kwargs):
        super(RewardShaped, self).__init__(**kwargs)
        
    def step(self, action):
        assert self.lander is not None

        # Update wind
        assert self.lander is not None, "You forgot to call reset()"
        if self.enable_wind and not (
            self.legs[0].ground_contact or self.legs[1].ground_contact
        ):
            # the function used for wind is tanh(sin(2 k x) + sin(pi k x)),
            # which is proven to never be periodic, k = 0.01
            wind_mag = (
                math.tanh(
                    math.sin(0.02 * self.wind_idx)
                    + (math.sin(math.pi * 0.01 * self.wind_idx))
                )
                * self.wind_power
            )
            self.wind_idx += 1
            self.lander.ApplyForceToCenter(
                (wind_mag, 0.0),
                True,
            )

            # the function used for torque is tanh(sin(2 k x) + sin(pi k x)),
            # which is proven to never be periodic, k = 0.01
            torque_mag = math.tanh(
                math.sin(0.02 * self.torque_idx)
                + (math.sin(math.pi * 0.01 * self.torque_idx))
            ) * (self.turbulence_power)
            self.torque_idx += 1
            self.lander.ApplyTorque(
                (torque_mag),
                True,
            )

        if self.continuous:
            action = np.clip(action, -1, +1).astype(np.float32)
        else:
            assert self.action_space.contains(
                action
            ), f"{action!r} ({type(action)}) invalid "

        # Engines
        tip = (math.sin(self.lander.angle), math.cos(self.lander.angle))
        side = (-tip[1], tip[0])
        dispersion = [self.np_random.uniform(-1.0, +1.0) / SCALE for _ in range(2)]

        m_power = 0.0
        if (self.continuous and action[0] > 0.0) or (
            not self.continuous and action == 2
        ):
            # Main engine
            if self.continuous:
                m_power = (np.clip(action[0], 0.0, 1.0) + 1.0) * 0.5  # 0.5..1.0
                assert m_power >= 0.5 and m_power <= 1.0
            else:
                m_power = 1.0
            # 4 is move a bit downwards, +-2 for randomness
            ox = tip[0] * (4 / SCALE + 2 * dispersion[0]) + side[0] * dispersion[1]
            oy = -tip[1] * (4 / SCALE + 2 * dispersion[0]) - side[1] * dispersion[1]
            impulse_pos = (self.lander.position[0] + ox, self.lander.position[1] + oy)
            p = self._create_particle(
                3.5,  # 3.5 is here to make particle speed adequate
                impulse_pos[0],
                impulse_pos[1],
                m_power,
            )  # particles are just a decoration
            p.ApplyLinearImpulse(
                (ox * MAIN_ENGINE_POWER * m_power, oy * MAIN_ENGINE_POWER * m_power),
                impulse_pos,
                True,
            )
            self.lander.ApplyLinearImpulse(
                (-ox * MAIN_ENGINE_POWER * m_power, -oy * MAIN_ENGINE_POWER * m_power),
                impulse_pos,
                True,
            )

        s_power = 0.0
        if (self.continuous and np.abs(action[1]) > 0.5) or (
            not self.continuous and action in [1, 3]
        ):
            # Orientation engines
            if self.continuous:
                direction = np.sign(action[1])
                s_power = np.clip(np.abs(action[1]), 0.5, 1.0)
                assert s_power >= 0.5 and s_power <= 1.0
            else:
                direction = action - 2
                s_power = 1.0
            ox = tip[0] * dispersion[0] + side[0] * (
                3 * dispersion[1] + direction * SIDE_ENGINE_AWAY / SCALE
            )
            oy = -tip[1] * dispersion[0] - side[1] * (
                3 * dispersion[1] + direction * SIDE_ENGINE_AWAY / SCALE
            )
            impulse_pos = (
                self.lander.position[0] + ox - tip[0] * 17 / SCALE,
                self.lander.position[1] + oy + tip[1] * SIDE_ENGINE_HEIGHT / SCALE,
            )
            p = self._create_particle(0.7, impulse_pos[0], impulse_pos[1], s_power)
            p.ApplyLinearImpulse(
                (ox * SIDE_ENGINE_POWER * s_power, oy * SIDE_ENGINE_POWER * s_power),
                impulse_pos,
                True,
            )
            self.lander.ApplyLinearImpulse(
                (-ox * SIDE_ENGINE_POWER * s_power, -oy * SIDE_ENGINE_POWER * s_power),
                impulse_pos,
                True,
            )

        self.world.Step(1.0 / FPS, 6 * 30, 2 * 30)

        pos = self.lander.position
        vel = self.lander.linearVelocity
        state = [
            (pos.x - VIEWPORT_W / SCALE / 2) / (VIEWPORT_W / SCALE / 2),
            (pos.y - (self.helipad_y + LEG_DOWN / SCALE)) / (VIEWPORT_H / SCALE / 2),
            vel.x * (VIEWPORT_W / SCALE / 2) / FPS,
            vel.y * (VIEWPORT_H / SCALE / 2) / FPS,
            self.lander.angle,
            20.0 * self.lander.angularVelocity / FPS,
            1.0 if self.legs[0].ground_contact else 0.0,
            1.0 if self.legs[1].ground_contact else 0.0,
        ]
        assert len(state) == 8

        reward = 0
        shaping = (
            -100 * np.sqrt(state[0] * state[0] + state[1] * state[1])
            - 100 * np.sqrt(state[2] * state[2] + state[3] * state[3])
            - 10 * abs(state[4])
            - 10 * abs(state[5])
            + 35 * state[6]
            + 35 * state[7]
        )  # And ten points for legs contact, the idea is if you
        # lose contact again after landing, you get negative reward
        if self.prev_shaping is not None:
            reward = shaping - self.prev_shaping
        self.prev_shaping = shaping

        reward -= (
            m_power * 0.15
        )  # less fuel spent is better, about -30 for heuristic landing
        reward -= s_power * 0.015

        terminated = False
        if self.game_over or abs(state[0]) >= 1.0:
            terminated = True
            reward = -100
        if not self.lander.awake:
            terminated = True
            reward = +150

        if self.render_mode == "human":
            self.render()
        return np.array(state, dtype=np.float32), reward, terminated, False, {}

In [172]:
from gym.envs.registration import register

In [173]:
register(
    id='Lon', # name given to this new environment
    entry_point=__name__+':RewardShaped', # env entry point
    kwargs={}
)

c:\Users\Dell\miniconda3\envs\gyenv\lib\site-packages\gym\envs\registration.py:498: UserWarning: WARN: Overriding environment Lon already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [174]:
from gym import envs
print(envs.registry["Lon"])

EnvSpec(id='Lon', entry_point='__main__:RewardShaped', reward_threshold=None, nondeterministic=False, max_episode_steps=None, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='Lon', version=None)


In [175]:
env = gym.make("Lon")

In [176]:
"""for i in range(10):
    total_rewards = 0
    observation, info = env.reset()
    done = False

    while not done:
        action = env.action_space.sample()  # agent policy that uses the observation and info
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_rewards += reward

        if done:
            break
        else:
            observation = new_state
    print(f"Episode {i}: Reward {total_rewards}")
    env.close()"""

'for i in range(10):\n    total_rewards = 0\n    observation, info = env.reset()\n    done = False\n\n    while not done:\n        action = env.action_space.sample()  # agent policy that uses the observation and info\n        new_state, reward, terminated, truncated, info = env.step(action)\n        done = terminated or truncated\n        total_rewards += reward\n\n        if done:\n            break\n        else:\n            observation = new_state\n    print(f"Episode {i}: Reward {total_rewards}")\n    env.close()'

In [177]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

In [178]:
import pennylane as qml

In [179]:
def ActorLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        #"y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, z_weights):
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            #for wire, y_weight in enumerate(y_weights[layer_idx]):
            #    qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits-1):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(6) @ qml.PauliZ(7)),
            qml.expval(qml.PauliZ(0) @ qml.PauliZ(2)),
            qml.expval(qml.PauliZ(1) @ qml.PauliZ(3)),
            qml.expval(qml.PauliZ(4) @ qml.PauliZ(5)) 
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [180]:
class Actor(nn.Module):
    def __init__(self, n_layers=5):
        super(Actor, self).__init__()
        self.n_qubits = 8
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.action_head = ActorLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.action_head(state)
        outputs = -1 + (outputs - outputs.min()) * (2) / (outputs.max() - outputs.min())
        distribution = Categorical(F.softmax(outputs, dim=-1))
        return distribution


"""class Critic(nn.Module):
    def __init__(self, n_layers=4):
        super(Critic, self).__init__()
        self.n_qubits = 4
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.critic_head = CriticLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.critic_head(state)
        return outputs"""

class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

In [181]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns



In [182]:

def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters(), lr = 0.005, amsgrad = True)
    optimizerC = optim.Adam(critic.parameters(), lr = 0.005, amsgrad = True)
    rewards_p = []
    returns_p = []
    for iter in range(n_iters):
        state, _ = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()
        total_rewards = 0
        done = False

        while not done:
            #env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, term, trunc, _ = env.step(action.cpu().numpy())
            done = term or trunc

            total_rewards += reward
            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, total_rewards))
                break

        rewards_p.append(total_rewards)
        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)
        returns_p.append(sum(returns))

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor.state_dict(), 'actorql2.pkl')
    torch.save(critic.state_dict(), 'criticql2.pkl')
    env.close()
    return rewards_p, returns_p


In [183]:
actor = Actor().to(device)

critic = Critic(state_size, action_size).to(device)


In [184]:
rewards, returns = trainIters(actor, critic, n_iters=1000)

Iteration: 0, Score: -94.06217231736818
Iteration: 1, Score: -82.09002120362042
Iteration: 2, Score: -41.98658692581627
Iteration: 3, Score: -73.28612425980114
Iteration: 4, Score: -91.90816879582488
Iteration: 5, Score: -65.256030903761
Iteration: 6, Score: -70.25238010086437
Iteration: 7, Score: -134.92376515914805
Iteration: 8, Score: -102.10965317920508
Iteration: 9, Score: -71.11471250139384
Iteration: 10, Score: -77.62232522124802
Iteration: 11, Score: -107.94013075650001
Iteration: 12, Score: -103.6362395549798
Iteration: 13, Score: -82.47839517462826
Iteration: 14, Score: -94.14654705695693
Iteration: 15, Score: -46.763386541287765
Iteration: 16, Score: -120.24015061719999
Iteration: 17, Score: -104.4148164587578
Iteration: 18, Score: -93.64626579155056
Iteration: 19, Score: -80.21741507234518
Iteration: 20, Score: -95.25270261104055
Iteration: 21, Score: -83.58884017450686
Iteration: 22, Score: 53.03203069739462
Iteration: 23, Score: -149.29056465855433
Iteration: 24, Score: -

In [185]:
import imageio

def saveanimation(frames,address="./lra2c.gif"):
    imageio.mimwrite(address, frames)

In [186]:
env1 = gym.make("Lon", render_mode = "rgb_array")

In [189]:
rewards = []
frames = []

for i in range(5):
    done = False
    t_rewards = 0
    state, info = env1.reset()
    while not done:
        frames.append(env1.render())
        state = torch.FloatTensor(state).to(device)
        dist, value = actor(state), critic(state)
        
        action = dist.sample()
        obs, reward, term, trunc, info = env1.step(action.cpu().numpy())
        done = term or trunc

        t_rewards += reward
        if done:
            break
        else:
            state = obs
    
    rewards.append(t_rewards)
    print(f"Episode {i}: Reward {t_rewards}")

saveanimation(frames)
env1.close()

Episode 0: Reward -14.732856710814715
Episode 1: Reward -8.71321991382564
Episode 2: Reward -77.31060021709703
Episode 3: Reward -30.028048614605993
Episode 4: Reward -29.76779075922265


In [190]:
torch.save(actor.state_dict(), "lra2ca.pt")
torch.save(critic.state_dict(), "lra2cc.pt")
